#### 코루틴(coroutine) Cooperative Routine 서로 협력 하는 루틴
#### 메인 루틴과 서브 루틴처럼 종속된 관계가 아님
#### 서로 대등한 관계이며 특정 시점에 상대방의 코드를 실행

## 41.1 코루틴에 값 보내기

#### 코루틴은 제너레이터의 특별한 형태
#### 제너레이터는 yield로 값을 발생 코루틴은 yield로 값을 받아올 수 있음
#### 코루틴에 값을 보내면서 코드를 실행할 때는 send 메서드를 사용
#### send메서드가 보낸 값을 받아오려면 (yield)형식으로 괄호로 묶어준 뒤 변수에 저장

In [5]:
def number_coroutine():
    while True:     # 코루틴을 계속 유지하기 위해 무한 루프 사용
        x = (yield) # 코루틴 바깥에서 값을 받아옴, yield를 괄호로 묶어야 함
        print(x)
        
co = number_coroutine()
next(co) # 코루틴 안의 yield까지 코드 실행(최초 실행)

co.send(1)
co.send(2)
co.send(3)

1
2
3


---

## 41.2 코루틴 바깥으로 값 전달하기

#### (yield 변수)형식으로 yield에 변수를 지정한 뒤 괄호로 묶어주면
#### 값을 받아오면서 바깥으로 값을 전달

#### 변수 = (yield 변수)
#### 변수 = next(코루틴객체)
#### 변수 = 코루틴객체.send(값)

In [6]:
def sum_coroutine():
    total = 0
    while True:
        x = (yield total) # 코루틴 바깥에서 값을 받아오면서 바깥으로 값을 전달
        total += x
        
co = sum_coroutine()
print(next(co))  # 0 : 코루틴 안의 yield까지 코드를 실행하고 코루틴에서 나온 값 출력

print(co.send(1)) # 1 : 코루틴에 숫자 1을 보내고 코루틴에서 나온 값 출력
print(co.send(2)) # 3 : 코루틴에 숫자 2를 보내고 코루틴에서 나온 값 출력
print(co.send(3)) # 6 : 코루틴에 숫자 3을 보내고 코루틴에서 나온 값 출력

0
1
3
6


#### 제너레이터는 next 함수를 반복 호출하여 값을 얻어내는 방식
#### 코루틴은 next함수를 한 번만 호출한 뒤 send로 값을 주고 받는 방식

---

## 41.3 코루틴을 종료하고 예외 처리하기

#### 코루틴은 실행 상태를 유지하기 위해 무한루프를 사용함
#### 종료하기 원하면 close 메서드를 사용

In [7]:
def number_coroutine():
    while True:
        x = (yield)
        print(x, end=" ")
        
co = number_coroutine()
next(co)

for i in range(20):
    co.send(i)
    
co.close() # 코루틴 종료

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 

#### 코루틴의 종료 시점을 알아야 할때 close 사용 시 편리

### 41.3.1 GeneratorExit 예외 처리하기

#### 코루틴 객체에서 close 메서드를 호출하면 코루틴이 종료될 때 GeneratorExit 예외가 발생

In [11]:
def number_coroutine():
    try:
        while True:
            x = (yield)
            print(x, end=' ')
    except GeneratorExit: # 코루틴이 종료 될 때 GeneratorExit 예외 발생
        print()
        print('코루틴 종료')
        
co = number_coroutine()
next(co)

for i in range(20):
    co.send(i)
    
co.close()

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
코루틴 종료


### 41.3.2 코루틴 안에 예외 발생시키기 

#### 코루틴 안에 예외 발생시킬 때 throw 메서드를 사용
#### 예외를 코루틴 안에 던짐
#### throw메서드에 지정한 에러 메시지 except as의 변수에 들어감
#### 코루틴객체.throw(예외이름, 에러메시지)

In [12]:
# 코루틴에 숫자 누적 RuntimeError 예외 발생시 에러 메시지를 출력 
# 누적된 값을 코루틴 바깥으로 전달

def sum_coroutine():
    try:
        total = 0
        while True:
            x = (yield)
            total += x
    except RuntimeError as e:
        print(e)
        yield total # 코루틴 바깥으로 값 전달
        
        
co = sum_coroutine()
next(co)

for i in range(20):
    co.send(i)

    
print(co.throw(RuntimeError, '예외로 코루틴 끝내기')) # 190
                                    # 코루틴의 except에서 yield로 전달 받은 값

예외로 코루틴 끝내기
190


---

## 41.4 하위 코루틴의 반환값 가져오기

#### 제너레이터에서 yield from을 사용하면 값을 바깥으로 여러 번 전달
#### 코루틴에서는 다르게 사용
#### yield from에 코루틴을 지정하면 해당 코루틴에서 return으로 반환 값을 가져옴

#### 변수 = yield from 코루틴()

In [13]:
def accumulate():
    total = 0
    while True:
        x = (yield) # 코루틴 바깥에서 값을 받아옴
        if x is None: # 받아온 값이 None이면
            return total # 합계 total을 반환
        total += x

def sum_coroutine():
    while True:
        total = yield from accumulate() # accumulate의 반환값을 가져옴
        print(total)
        
co = sum_coroutine()
next(co)

for i in range(1, 11):  # 1부터 10까지 반복
    co.send(i)          # 코루틴 accumulate에 숫자를 보냄
co.send(None)          # 코루틴 accumulate에 NOne을 보내서  숫자 누적을 끝냄

for i in range(1, 101): # 1부터 100까지 반복
    co.send(i)          # 코루틴 accumulate에 숫자를 보냄
co.send(None)          # 코루틴 accumulate에 None을 보내서 숫자 누적을 끝냄

55
5050


### 41.4.1 StopIteration 예외 발생시키기

#### 코루틴도 제너레이터이므로 return을 사용하면 StopIteration 예외 발생
#### 코루틴에서 return 값은 raise StopIteration(값)처럼 사용할 수도 있음(python 3.6 이하)
#### 3.7부터는 그냥 return 값을 사용 (RuntimeError로 바뀜)

#### raise StopIteration(값)

In [15]:
!python -V

Python 3.9.12


In [18]:
def accumulate():
    total = 0
    while True:
        x = (yield)                    # 코루틴 바깥에서 값을 받아옴
        if x is None:                 # 받아온 값이 None이면
            raise StopIteration(total)# StopIteration에 반환할 값을 지정(python 3.6 이하)
            return total
        total += x


def sum_coroutine():
    while True:
        total = yield from accumulate() # accumulate의 반환값을 가져옴
        print(total)
        
        
co = sum_coroutine()
next(co)

for i in range(1, 11):  # 1부터 10까지 반복
    co.send(i)          # 코루틴 accumulate에 숫자를 보냄
co.send(None)          # 코루틴 accumulate에 NOne을 보내서  숫자 누적을 끝냄

for i in range(1, 101): # 1부터 100까지 반복
    co.send(i)          # 코루틴 accumulate에 숫자를 보냄
co.send(None)          # 코루틴 accumulate에 None을 보내서 숫자 누적을 끝냄    

RuntimeError: generator raised StopIteration

In [19]:
def accumulate():
    total = 0
    while True:
        x = (yield)                    # 코루틴 바깥에서 값을 받아옴
        if x is None:                 # 받아온 값이 None이면
#             raise StopIteration(total)# StopIteration에 반환할 값을 지정(python 3.6 이하)
            return total
        total += x


def sum_coroutine():
    while True:
        total = yield from accumulate() # accumulate의 반환값을 가져옴
        print(total)
        
        
co = sum_coroutine()
next(co)

for i in range(1, 11):  # 1부터 10까지 반복
    co.send(i)          # 코루틴 accumulate에 숫자를 보냄
co.send(None)          # 코루틴 accumulate에 NOne을 보내서  숫자 누적을 끝냄

for i in range(1, 101): # 1부터 100까지 반복
    co.send(i)          # 코루틴 accumulate에 숫자를 보냄
co.send(None)          # 코루틴 accumulate에 None을 보내서 숫자 누적을 끝냄    

55
5050


#### 코루틴의 yield from으로 값을 발생시키기

In [21]:
def number_coroutine():
    x = None
    while True:
        x = (yield x) # 코루틴 바깥에서 값을 받아오면서 바깥으로 값을 전달
        if x == 3:
            return x
        
def print_coroutine():
    while True:
        x = yield from number_coroutine() # 하위 코루틴의 yield에 지정된 값을 다시 바깥으로 전달
        print('print_coroutine', x)
        
co = print_coroutine()
next(co)

x = co.send(1) # number_coroutine으로 1을 보냄
print(x)       # 1 : number_coroutine의 yield에서 바깥으로 전달한 값
x = co.send(2) # number_coroutine으로 2를 보냄
print(x)       # 2 : number_coroutine의 yield에서 바깥으로 전달한 값
co.send(3)     # 3을 보내서 반환값을 출력하도록 만듬

1
2
print_coroutine 3
